# EDA

_Francisco José Álvarez Rojo_

_Jorge III Altamirano Astorga - 175904_

_Paulina Gómez Mont Wickers_

In [18]:
from bs4 import BeautifulSoup
import bs4 as bs4
import requests
import sys
from dateutil import parser #esta lib es más útil que la de Python para interpretar fechas parciales, como las de Twitter
import re
import datetime
import pandas as pd
from string import punctuation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import word_tokenize
# from nltk.sentiment import SentimentAnalyzer
from sklearn.naive_bayes import MultinomialNB


## Lectura de los Datos

### Tweets

Pasamos los tweets a un Pandas Dataframe con el fin poder manipular mejor la información.

In [2]:
bmv = pd.read_csv("tweets.full/elfinanciero_mx.2014.csv", sep="|") 
for i in range(2015,2020):
    bmv = bmv.append(pd.read_csv("tweets.full/elfinanciero_mx.%d.csv"%i, sep="|")) 
bmv["source"] = "elfinanciero_mx"
for i in range(2014,2020):
    bmv = bmv.append(pd.read_csv("tweets.full/eleconomista.%d.csv"%i, sep="|")) 
bmv["source"] = np.where(pd.isna(bmv["source"]), "eleconomista", bmv.source)
for i in range(2014,2020):
    bmv = bmv.append(pd.read_csv("tweets.full/eluniversal.%d.csv"%i, sep="|")) 
bmv["source"] = np.where(pd.isna(bmv["source"]), "eluniversal", bmv.source)
bmv.time = pd.to_datetime(bmv.time)
bmv.tweet = bmv.tweet.astype("str")
bmv.source = bmv.source.astype("str")
bmv = bmv.drop(columns=["page"])
bmv.to_csv("tweets.csv.xz", compression="xz")
bmv.dtypes

/home/jaa6766/.local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


source            object
time      datetime64[ns]
tweet             object
dtype: object

In [3]:
try:
    del(bmv) #borrar variable, para no ocupar memoria
except:
    ;
tweets = pd.read_csv("tweets.csv.xz", compression="xz")
tweets.time = pd.to_datetime(tweets.time)
#ordenar columnas 
tweets = tweets.sort_values(by=["time"])
#remover tweets con fechas a futuro (error de interpretación de fechas)
tweets = tweets[tweets.time < datetime.date.today().isoformat()] 
#reindizar
tweets.index = range(0, tweets.shape[0])
tweets.head(3)

,Unnamed: 0,source,time,tweet
0,26948,eluniversal,2013-12-31 20:14:00,"Se acaba el año, por eso conoce la noticias má..."
1,26946,eluniversal,2013-12-31 20:14:00,Times Square congregá a 1 millón de personas p...
2,26947,eluniversal,2013-12-31 20:14:00,#PORSINOLOVISTE @inah_mx halló cráneos de tzom...


In [4]:
tweets.shape

(356058, 4)

### IPC

In [5]:
ipc = pd.read_csv("bmv_ipc.csv", sep=',')
ipc.Date = pd.to_datetime(ipc.Date)
#quitamos columnas menos relevantes para nuestro análisis
ipc = ipc.drop(["Open", "High", "Low", "Close"], axis=1)
#quito espacios
ipc = ipc.rename({"Adj Close": "close", "Volume": "volume", "Date": "date"}, axis="columns")
#sacamos la diferencia porcentual del cierre del día anterior
ipc["deltaPct"] = 100 * (1 - ipc["close"].shift(1) / ipc["close"])
#asignamos un sentimiento basado en el cambio
ipc.loc[ipc.deltaPct <= -0.5, "sentiment"] = -1 #sentimiento negativo
ipc.loc[ipc.deltaPct >= +0.5, "sentiment"] = 1 #sentimiento positivo
ipc.loc[ipc.sentiment.isna(), "sentiment"] = 0 #sentimiento neutral
#en caso de no existir sentimiento lo ponemos neutral
ipc.loc[ipc.deltaPct.isna(), "deltaPct"] = 0
print(ipc.head())
ipc.dtypes

        date         close       volume  deltaPct  sentiment
0 2013-12-26  42540.320313  139436700.0  0.000000        0.0
1 2013-12-27  42753.218750  154856000.0  0.497971        0.0
2 2013-12-30  42958.820313  175830000.0  0.478602        0.0
3 2014-01-02  42188.449219  139707400.0 -1.826024       -1.0
4 2014-01-03  42064.968750  141022700.0 -0.293547        0.0


date         datetime64[ns]
close               float64
volume              float64
deltaPct            float64
sentiment           float64
dtype: object

In [6]:
print(ipc.date.min(), ipc.date.max())

2013-12-26 00:00:00 2019-05-14 00:00:00


In [7]:
print(tweets.time.min(), tweets.time.max())

2013-12-31 20:14:00 2019-05-13 00:00:00


## NLP

In [8]:
stop_words = stopwords.words('spanish')
stemmer = nltk.stem.SnowballStemmer('spanish')
non_words = list(punctuation)
non_words.extend(['¿', '¡'])

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
def tokenize(text):
    text = ''.join([c for c in text if c not in non_words])
    tokens = word_tokenize(text)
    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

De manera experimental realizaremos un entrenamiento con el sentimiento que obtuvimos en el paso anterior (IPC), y lo asignaremos a los valores de la misma fecha del tweet.

In [9]:
tweets.drop(columns=["sentiment"], inplace=True, errors="ignore")
#sentimiento basado en el porcentaje de cambio del ipc
tweets["ipc"] = tweets.join(ipc.set_index("date"), on="time", how="left").deltaPct.round(0)
#sentimiento basado en reglas
tweets["sentiment"] = tweets.join(ipc.set_index("date"), on="time", how="left").sentiment
tweets.drop(tweets.filter(regex="Unnamed"), axis=1, inplace=True) #remover índice incorrecto
tweets.head(10)

,source,time,tweet,ipc,sentiment
0,eluniversal,2013-12-31 20:14:00,"Se acaba el año, por eso conoce la noticias má...",NaN,NaN
1,eluniversal,2013-12-31 20:14:00,Times Square congregá a 1 millón de personas p...,NaN,NaN
2,eluniversal,2013-12-31 20:14:00,#PORSINOLOVISTE @inah_mx halló cráneos de tzom...,NaN,NaN
3,eluniversal,2013-12-31 20:14:00,#FOTOGALERÍA Hallan cráneos en Línea 12 del M...,NaN,NaN
4,eluniversal,2013-12-31 20:14:00,De Blasio jura mañana como nuevo alcalde de Nu...,NaN,NaN
5,eluniversal,2013-12-31 20:14:00,#ENTÉRATE Hospitalizan a ex primera dama de #E...,NaN,NaN
6,eluniversal,2013-12-31 20:14:00,#FOTO Miles de personas esperan el 2014 en Tim...,NaN,NaN
7,eluniversal,2013-12-31 20:14:00,Desmontan estatua de Stalin en su Georgia nata...,NaN,NaN
8,eluniversal,2013-12-31 20:14:00,"Gentleman de Psy, el video más visto de 2013 e...",NaN,NaN
9,eluniversal,2013-12-31 20:14:00,Reportan en Tlaxcala dos muertos por influenza...,NaN,NaN


Existen fechas en las cuales no opera la bolsa, pero sí existen tweets publicados; por lo que es conveniente aplicar dicho tweet al cierre de la jornada del siguiente día hábil para la bolsa. Se puede observar en la salida anterior, el caso de un 1 de Enero, donde la bolsa no laboró. Pero el 2 de enero sí, entonces, dicho tweet aplica para el 2 de Enero. 

Lo mismo aplica para tweets que son publicados en Sábado o Domingo, por lo que aplicamos el sentimiento del siguiente día hábil.

In [10]:
def tweet_adjust_df(df, columns=["sentiment", "ipc"], ipc=ipc[["date", "sentiment", "deltaPct"]]):
    """Ajusta el dataframe de tweets con los sentimientos y valor IPC del siguiente día hábil.
    Ojo: el dataframe del IPC deberá conteneter (en orden):
    * fecha: es con lo que hacemos el índice
    * los siguientes valores que coincidan con el parámetro `columns`
    """
    df = df.copy()
    ipc = ipc.copy()
    i = 0
    for col in columns:
        for (idx, row) in df.loc[df[col].isna(), :].iterrows():
            #buscar en las siguientes fechas hábiles, 4 días máx
            dates = [df.loc[idx, "time"] + datetime.timedelta(days=j) for j in range (0,6)]
            dates = ['{0:%Y-%m-%d}'.format(datum) for datum in dates]
            ipc_dates = ipc.loc[ipc.date.isin(dates), ipc.keys()[i+1]].head(1)
        i += 1
    #establecer en sentimiento neutral (0) los que aún así no pudimos determinar el sentimiento
    df.loc[np.any(df.isna(), axis=1), np.any(df.isna(), axis=0)] = 0
    return df
tweets = tweet_adjust_df(tweets, ["ipc", "sentiment"]).copy()
# tweets = tweets.drop(columns=["Unnamed: 0"])
tweets.head(3)

,source,time,tweet,ipc,sentiment
0,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
1,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
2,eluniversal,2013-12-31 20:14:00,0,0.0,0.0


In [11]:
keywords = ['america movil','AMX','banco de mexico','banxico', 'mexico', 'bmv', 
            'bolsa mexicana de valores', 'ipc', 'walmex','femsa','kof',
            'televisa', 'TLEVISA', 'grupo mexico', 'gmexico', 'banorte','GFNORTEO',
            'cemex','alfa', 'peñoles', 'pe&oles', 'inbursa', "DINBUR1A",
            'elektra', 'mexichem', 'MEXCHEM', 'bimbo', 'arca continental', 'ac',
            'kimberly-clark', "KIMBERA", 'genomma lab', 'labb',
            'puerto de liverpool', 'LIVEPOLC', 'grupo aeroportuario', "ASURB", 
            "omab", 'alpek', 'ica', 'tv azteca', 'AZTECA',
            'ohl', 'maseca', 'GRUMA', 'alsea', 'carso', 'lala', 'banregio',
            'ienova', 'pinfra', 'santander mexico', 'presidente de mexico', "BSMXB", 
            'cetes', 'tasa']
keywords[:3]

['america movil', 'AMX', 'banco de mexico']

In [12]:
tweets.head()

,source,time,tweet,ipc,sentiment
0,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
1,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
2,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
3,eluniversal,2013-12-31 20:14:00,0,0.0,0.0
4,eluniversal,2013-12-31 20:14:00,0,0.0,0.0


In [16]:
tweets[tweets.tweet.str.contains("0")]

ValueError: cannot index with vector containing NA / NaN values